In [2]:
# https://man.twcc.ai/@TWCC-III-manual/ryyo0tsuu
def SLURM(cmd):
    ## SLURM 內容, 請修改 ---> Email
    SLURM='''#!/work/c00cjz002/binary/bash5.0/bin/bash
#SBATCH -A GOV110079        # 計畫代號
#SBATCH -J CPU_T3_C2        # 工作代號 (標籤, 可自行定義)
#SBATCH -p ct56             # 工作區塊 
#SBATCH -c 3                # 使用的CPU核心數
##SBATCH --mem=12g           # 使用的記憶體容量 (不設定自動給予ram 10g)
#SBATCH --mail-user=summerhill001@gmail.com    # 請修改為您的信向
#SBATCH --mail-type=BEGIN,END                  # 指定送出email時機 可為NONE, BEGIN, END, FAIL, REQUEUE, ALL
#SBATCH -o log/%j.logi      # 執行記錄檔案儲存於log目錄底下
'''
    myCmd = SLURM + cmd
    
    ## 儲存上述內容 SLURM.sh
    import time
    slurm_shell = 'slurm/'+time.strftime("%Y-%m-%d_%H-%M-%S")+'.sh'
    
    f = open(slurm_shell, "w")
    f.write(myCmd)
    f.close()    

    ## 執行SLURM
    #!sbatch SLURM.sh
    jobID=(subprocess.check_output('sbatch '+slurm_shell+' |awk \'{print $4}\'', shell=True,text=True))
    return jobID

## 建立目錄
import subprocess
!mkdir -p slurm
!mkdir -p log

In [3]:
def createCMD(SUBJECTS_DIR, inputFile, saveFolder):
    cmd='''
echo "這是測試頁面09"

# -> 啟動 jupyter
~/.nchc_jupyter/startifdown_jupyterlab.sh >> ~/jupyter.log 

# -> 環境變數 1
export FREESURFER_HOME=/work/c00cjz002/package/freesurfer711
source $FREESURFER_HOME/SetUpFreeSurfer.sh

# -> 環境變數 2 (請自行修改)
SUBJECTS_DIR='''+ SUBJECTS_DIR +''' 
inputFile='''+ inputFile +''' 
saveFolder='''+ saveFolder +''' 

# -> 切換目錄
export SUBJECTS_DIR='''+ SUBJECTS_DIR +''' 
mkdir -p $SUBJECTS_DIR
cd $SUBJECTS_DIR

# -> 執行程式
echo "STEP01"
recon-all -s $saveFolder -i $inputFile -all -qcache -parallel -openmp 3
echo "STEP02"
segmentThalamicNuclei.sh $saveFolder $FREESURFER_HOME
echo "STEP03"
segmentHA_T1.sh $saveFolder $FREESURFER_HOME
echo "STEP04"
mri_convert $saveFolder/mri/aparc+aseg.mgz $saveFolder/aparc+aseg.nii --out_orientation LAS
echo "STEP05"
mri_convert $saveFolder/mri/orig.mgz $saveFolder/orig.nii --out_orientation LAS
echo "STEP06"
mri_convert $saveFolder/mri/wmparc.mgz $saveFolder/wmparc.nii --out_orientation LAS
echo "STEP07"
mri_convert $saveFolder/mri/lh.hippoAmygLabels-T1.v21.FSvoxelSpace.mgz $saveFolder/lh.hippoAmygLabels-T1.v21.FSvoxelSpace.nii --out_orientation LAS
echo "STEP08"
mri_convert $saveFolder/mri/lh.hippoAmygLabels-T1.v21.FSvoxelSpace.mgz $saveFolder/rh.hippoAmygLabels-T1.v21.FSvoxelSpace.nii --out_orientation LAS
echo "STEP09"
mri_convert $saveFolder/mri/ThalamicNuclei.v12.T1.FSvoxelSpace.mgz $saveFolder/ThalamicNuclei.v12.T1.FSvoxelSpace.nii --out_orientation LAS
echo "STEP10"
zip -r $saveFolder.zip $saveFolder
'''
    return cmd

In [4]:
## 指令集
import os
import re
from os import walk
SUBJECTS_DIR="${HOME}/freesurfer/demo/"
FILES_PATH="/work/c00cjz002/data/Powei20211109/"
files = next(walk(FILES_PATH), (None, None, []))[2]  # [] if no file
pattern = "img$"
for file in files:
    if re.search(pattern, file):
        inputFile=FILES_PATH+file
        saveFolder=os.path.splitext(os.path.basename(inputFile))[0]
        cmd=createCMD(SUBJECTS_DIR, inputFile, saveFolder)
        print(cmd)
        #jobID = SLURM(cmd)
        #print(jobID.strip()+' '+saveFolder)


echo "這是測試頁面05"

# -> 啟動 jupyter
~/.nchc_jupyter/startifdown_jupyterlab.sh >> ~/jupyter.log 


# -> 主要程式
export FREESURFER_HOME=/work/c00cjz002/package/freesurfer711
source $FREESURFER_HOME/SetUpFreeSurfer.sh
export SUBJECTS_DIR=${HOME}/freesurfer/demo/ 
SUBJECTS_DIR=${HOME}/freesurfer/demo/ 
inputFile=/work/c00cjz002/data/Powei20211109/11477640.img 
saveFolder=11477640 
cd $SUBJECTS_DIR

echo "STEP01"
recon-all -s $saveFolder -i $inputFile -all -qcache -parallel -openmp 3


echo "這是測試頁面05"

# -> 啟動 jupyter
~/.nchc_jupyter/startifdown_jupyterlab.sh >> ~/jupyter.log 


# -> 主要程式
export FREESURFER_HOME=/work/c00cjz002/package/freesurfer711
source $FREESURFER_HOME/SetUpFreeSurfer.sh
export SUBJECTS_DIR=${HOME}/freesurfer/demo/ 
SUBJECTS_DIR=${HOME}/freesurfer/demo/ 
inputFile=/work/c00cjz002/data/Powei20211109/02856573.img 
saveFolder=02856573 
cd $SUBJECTS_DIR

echo "STEP01"
recon-all -s $saveFolder -i $inputFile -all -qcache -parallel -openmp 3


echo "這是測試頁面05"

# -> 啟動 jupyter
~/.nch

In [13]:
## 觀看執行狀況
!squeue -u `whoami` | grep "$jobID\|JOBID" 

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
            239069      ct56 CPU_T3_C c00cjz00  R       0:10      1 cpn3033 


In [ ]:
## 刪除JOB
!scancel $jobID

In [15]:
## 全部JOB刪除
!squeue -u `whoami` | grep -v JOBID  | awk '{print $1}' | xargs scancel  # 列出 PID 並砍掉 Process